# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 

### Associated lectures: [Lectures 17 and 18](https://github.com/UBC-CS/cpsc330-2024s/tree/main/lectures)

**See PrairieLearn for _due date_ and _submission_**

## Imports

In [1]:
import os
from hashlib import sha1

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>

_points: 2_

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size $\leq$ 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Be sure to follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024s/blob/main/docs/homework_instructions.md).
- Upload the .ipynb file to PrairieLearn.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 17, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [2]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [3]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [4]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings

_Points:_ 2

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

In [5]:
glove_wiki_vectors.most_similar("mathematics")

[('physics', 0.834094762802124),
 ('mathematical', 0.7631470561027527),
 ('science', 0.76076740026474),
 ('biology', 0.7574610710144043),
 ('chemistry', 0.7529067397117615),
 ('sociology', 0.7426297664642334),
 ('philosophy', 0.7377276420593262),
 ('humanities', 0.7304596900939941),
 ('linguistics', 0.7110267281532288),
 ('sciences', 0.7065612077713013)]

In [6]:
glove_wiki_vectors.most_similar("lasagna")

[('risotto', 0.7129864692687988),
 ('pasta', 0.6891202926635742),
 ('meatballs', 0.6853904724121094),
 ('meatloaf', 0.6844348907470703),
 ('ravioli', 0.6840620040893555),
 ('casserole', 0.6840508580207825),
 ('noodles', 0.6687648296356201),
 ('spaghetti', 0.6492099165916443),
 ('gnocchi', 0.6478359699249268),
 ('polenta', 0.646486222743988)]

In [7]:
glove_wiki_vectors.most_similar("vancouver")

[('calgary', 0.8236873149871826),
 ('toronto', 0.8086004853248596),
 ('edmonton', 0.7792404294013977),
 ('ottawa', 0.7779432535171509),
 ('winnipeg', 0.7616495490074158),
 ('montreal', 0.7592548131942749),
 ('portland', 0.6879022717475891),
 ('phoenix', 0.6766419410705566),
 ('seattle', 0.6594581604003906),
 ('philadelphia', 0.6500991582870483)]

In [8]:
glove_wiki_vectors.most_similar("ubc")

[('unm', 0.588445246219635),
 ('ucl', 0.5795673131942749),
 ('dalhousie', 0.5718241930007935),
 ('unsw', 0.5628497004508972),
 ('wvu', 0.541490375995636),
 ('optometry', 0.5404933094978333),
 ('bsu', 0.5389480590820312),
 ('uvm', 0.5379002690315247),
 ('vcu', 0.5366262793540955),
 ('fiu', 0.536444902420044)]

<!-- END QUESTION -->

<br><br>

### 1.2 Word similarity using pre-trained embeddings

_Points:_ 2

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [9]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

In [10]:
coast_shore_similarity = None
clothes_closet_similarity = None
old_new_similarity = None
smart_intelligent_similarity = None
dog_cat_similarity = None
tree_lawyer_similarity = None


coast_shore_similarity = glove_wiki_vectors.similarity(word_pairs[0][0], word_pairs[0][1])
clothes_closet_similarity = glove_wiki_vectors.similarity(word_pairs[1][0], word_pairs[1][1])
old_new_similarity = glove_wiki_vectors.similarity(word_pairs[2][0], word_pairs[2][1])
smart_intelligent_similarity = glove_wiki_vectors.similarity(word_pairs[3][0], word_pairs[3][1])
dog_cat_similarity = glove_wiki_vectors.similarity(word_pairs[4][0], word_pairs[4][1])
tree_lawyer_similarity = glove_wiki_vectors.similarity(word_pairs[5][0], word_pairs[5][1])

print(coast_shore_similarity)
print(clothes_closet_similarity)
print(old_new_similarity)
print(smart_intelligent_similarity)
print(dog_cat_similarity)

0.70002705
0.54627603
0.64324874
0.7552731
0.8798074


<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings

_Points:_ 6

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.

In [11]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [12]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [13]:
glove_wiki_vectors.similarity("white", "rich")

0.4472359

In [14]:
glove_wiki_vectors.similarity("black", "rich")

0.51745194

<div class="alert alert-warning">

Solution_1.3
    
</div>

_There are some biases exist as shown in the examples below. For example, in words that are similar to black as the same sense of educated similar to white, illiterate and gifted scores higher than well-educated and literate. Also, schooled and young is much higher than well-educated, which infers a difference in sense of socioeconomic status and quality of education. Another example is that in words that are similar to woman as the same sense of math similar to man, humanities subjects appear more often, such as sociology and linguistics. Another noticeable example is the huge gap between the similarity score of cisgender and disease and transgender and disease._

In [15]:
glove_wiki_vectors.similarity("male", "cooking")

0.13481157

In [16]:
glove_wiki_vectors.similarity("female", "cooking")

0.18214385

In [17]:
analogy("white", "educated", "black")

white : educated :: black : ?


,Analogy word,Score
0,young,0.619187
1,younger,0.577345
2,schooled,0.571257
3,graduates,0.569345
4,affluent,0.566490
5,illiterate,0.566026
6,gifted,0.564713
7,born,0.562217
8,well-educated,0.561142
9,literate,0.554493


In [18]:
analogy("man", "mathematics", "woman")

man : mathematics :: woman : ?


,Analogy word,Score
0,humanities,0.691818
1,sociology,0.688262
2,undergraduate,0.677993
3,biology,0.673875
4,physics,0.672595
5,sciences,0.664970
6,linguistics,0.655082
7,doctorate,0.647279
8,graduate,0.645335
9,phd,0.641917


In [19]:
glove_wiki_vectors.similarity("transgender", "disorder")

0.22772317

In [20]:
glove_wiki_vectors.similarity("cisgender", "disorder")

-0.081149615

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion

_Points:_ 4

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

_1. I'd say the quality is not very high as there are a lot of biases existing between difference gender and race._

_2. For example, when using this model on healthcare records or screening based on patients' information and past records, the model would be more likely to link the possibility of disorder with transgender patients than cisgender ones._

<!-- END QUESTION -->

<br><br>

### 1.5 Classification with pre-trained embeddings 

_Points:_ 8

In lecture 17, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under the data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`, be sure to filter out stop words), calculate the train and test accuracy and show a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.
2. Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show the [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.

In [21]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [22]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [23]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [24]:
import spacy
import en_core_web_md

# nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Type your answer here, replacing this text._

In [25]:
bow_train_accuracy = None
bow_test_accuracy = None
bow_report = None # use digits=2

from sklearn.metrics import classification_report


pipe = make_pipeline(
    CountVectorizer(stop_words="english"), LogisticRegression(max_iter=1000)
)
pipe.named_steps["countvectorizer"].fit(X_train)
X_train_transformed = pipe.named_steps["countvectorizer"].transform(X_train)
pipe.fit(X_train, y_train)
bow_train_accuracy = pipe.score(X_train, y_train)
bow_test_accuracy = pipe.score(X_test, y_test)
bow_report = classification_report(y_test, pipe.predict(X_test))
print(bow_train_accuracy)
print(bow_test_accuracy)
print(bow_report)

0.9558005461717407
0.8178386031146767
                  precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.55      0.57       469
        exercise       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.72       407
          nature       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.82      4238



In [26]:
X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])

In [27]:
# print(X_train_embeddings.shape)

In [28]:
lgr = LogisticRegression(max_iter=10000)
lgr.fit(X_train_embeddings, y_train)

LogisticRegression(max_iter=10000)

In [29]:
emb_train_accuracy = lgr.score(X_train_embeddings, y_train)
emb_test_accuracy = lgr.score(X_test_embeddings, y_test)
emb_report = classification_report(y_test, lgr.predict(X_test_embeddings)) # use digits=2
print(emb_train_accuracy)
print(emb_test_accuracy)
print(emb_report)

0.8503084858905634
0.7949504483246814
                  precision    recall  f1-score   support

     achievement       0.81      0.83      0.82      1302
       affection       0.86      0.91      0.89      1423
         bonding       0.83      0.77      0.80       492
enjoy_the_moment       0.57      0.51      0.54       469
        exercise       0.68      0.76      0.72        74
         leisure       0.72      0.65      0.68       407
          nature       0.70      0.72      0.71        71

        accuracy                           0.79      4238
       macro avg       0.74      0.74      0.74      4238
    weighted avg       0.79      0.79      0.79      4238



<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion

_Points:_ 6

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

1. Using `CountVectorizer`, the resulting vector contains elements of the count number of each unique word in a document. Whereas using average-embedding approach, each unique word is a vector and elements in the resulting vector is the average dense vectors of different words.
2. I'd say the `CountVectorizer` representation is more interpretable, as each element refers to a unique word in the training set. However, for average-embedding approach, we might not be able to interpret the average of dense vector (each element in the resulting vector) directly.
3. I think the average-embedding approach uses transfer learning. `CountVectorizer` purely uses training data, while the average-embedding relies on existed texts and trains on them first to create the embedding.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [30]:
from sklearn.datasets import fetch_20newsgroups

In [31]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [32]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)

_Points:_ 8

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

In [33]:
# helper function from Lecture 17
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """

    clean_text = []

    for token in doc:
        if (
            token.is_punct == False # check if it's not punctuation: comma and semicolon, question mark
            and token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos # check if it's not slashes, new-line characters, or any other non-informative characters
        ): 
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower()) # # change to lowercase
        elif token.like_url:  # Check if it's a url
            clean_text.append("URL")
        elif token.like_email: # check if it's an email
            clean_text.append("EMAIL")
        elif token.like_num: # check if it's a number
            clean_text.append("NUM")
    return " ".join(clean_text)

In [34]:
df["text_pp"] = [
    preprocess(spacy_text) for spacy_text in nlp.pipe(df["text"].tolist())
]

In [35]:
df["text_pp"]

0       know read NUM u.s.c. 922 sence wonder help u.s...
1       bad question ref list algorithm think bit hard...
2       actuallay hand support idea have NUM newsgroup...
3       follow problem bug appreciate help create NUM ...
4       late upi foreign ministry spokesman ferhat ata...
                              ...                        
4558    look software call shadow know simple raytrace...
4559    archive faq part3 modify 1993/04/04 subject NU...
4560    nice answer question difference fed mandate li...
4561    get gateway 4dx-33v try configure need correct...
4562    armenians nagarno karabagh defend right homela...
Name: text_pp, Length: 4563, dtype: object

In [36]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have NUM newsgroup...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create NUM ...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,like subscribe leadership magazine wonder NUM ...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification

_Points:_ 2

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps.

<div class="alert alert-warning">

Solution_2_2
    
</div>

1. word tokenization
2. getting rid of punctuation, stopword, and non-informative characters like slashes, because they are not informative and useful in topic modelling
3. getting rid of words with specific POS tags
4. replacing specific urls, email addresses, and numbers with "URL", "EMAIL", and "NUM", as their original form is too unique for text recognition and it's better if we could categorize them

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation

_Points:_ 4

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_I picked 4 as the number of topics for `n_components` in the end, because I think the data can be separated into four different themes: news, computer, Christian, and sports after 4 is picked._

In [37]:
vec = CountVectorizer()
vec.fit(df["text_pp"])
X_vec = vec.transform(df["text_pp"])

In [38]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 4 # number of topics
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=0
)
lda.fit(X_vec) 
document_topics = lda.transform(X_vec)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association

_Points:_ 5

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

In [39]:
# Code from Lecture 17
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

In [40]:
import mglearn

mglearn.tools.print_topics(
    topics=range(4),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
num           num           num           num           
people        file          god           game          
gun           program       know          team          
right         image         people        play          
state         use           think         year          
know          edu           say           player        
think         window        come          win           
israel        available     believe       season        
year          server        time          good          
time          run           thing         armenian      




<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association

_Points:_ 5

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

In [41]:
for n in range(0,5):
    print(df.iloc[n]['text'])
    print(document_topics[n])

You know, I was reading 18 U.S.C. 922 and something just did not make 
sence and I was wondering if someone could help me out.

Say U.S.C. 922 :

(1) Except as provided in paragraph (2), it shall be unlawful for
any person to transfer or possess a machinegun.

 Well I got to looking in my law dictionary and I found that a "person" 
might also be an artificial entity that is created by government 
and has no rights under the federal constitution. So, what I 
don't understand is how a statute like 922 can be enforced on 
an individual. So someone tell me how my government can tell
me what I can or cannot possess. Just passing a law 
does not make it LAW. Everyone knows that laws are constitional
until it goes to court. So, has it ever gone to court, not
just your run of the mill "Ok I had it I am guilty, put me in jail"

Has anyone ever claimed that they had a right to possess and was told
by the Supreme Court that they didn't have that right?



[0.98724799 0.00416158 0.0043501  0.00424

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

_Points:_ 6

1. In lecture 18, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 18 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

1. Decision tree, KNN, and RandomForest can handle multiclass classification. In tree-related models, it is common to use different nodes and leaves to represent multiple classes. In KNN, we can define multiple centre points at the same time to handle multiclass situation. Logistic Regression and SVM RBF might not extend naturally to multiclass classification. They may use the one-vs.-rest approach and the one-vs.-one approach instead.
2. Transfer learning is one of the most common computer vision and natural language processing method of downloading a pre-trained model and fine tuning it for the task.
3. Neural networks can be very hard to train and therefore if we have pre-trained networks, they can save much time and effort for us. We can directly use these pre-trained neural networks during transfer learning.

<!-- END QUESTION -->

<br><br><br><br>

## Submission instructions 

**PLEASE READ:** When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using PrairieLearn.
4. Make sure that the plots and output are rendered properly in your submitted file.